# Output Parser Patterns with Amazon Nova

This notebook demonstrates parsing and structuring LLM outputs using LangChain parsers.

## Setup

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

In [ ]:
from langchain_amazon_nova import ChatAmazonNova
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser,
    PydanticOutputParser,
)
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

# Initialize the model
llm = ChatAmazonNova(model="nova-pro-v1", temperature=0)

## 1. String Parser

The simplest parser - converts AIMessage to string.

In [ ]:
str_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")
chain = prompt | llm | str_parser

result = chain.invoke({"country": "Japan"})
print(f"Result: {result}")
print(f"Type: {type(result)}")

## 2. Comma-Separated List Parser

Parse comma-separated values into a list.

In [ ]:
list_parser = CommaSeparatedListOutputParser()

prompt = ChatPromptTemplate.from_template("List 5 {category}.\n{format_instructions}")

chain = prompt | llm | list_parser
result = chain.invoke(
    {
        "category": "programming languages",
        "format_instructions": list_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"First item: {result[0]}")

## 3. JSON Parser

Parse JSON output into Python dictionaries.

In [ ]:
json_parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_template(
    "Provide information about {topic} in JSON format with keys: name, description, year_created.\n{format_instructions}"
)

chain = prompt | llm | json_parser
result = chain.invoke(
    {
        "topic": "Python programming language",
        "format_instructions": json_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"Name: {result.get('name')}")

## 4. Pydantic Parser

Parse into strongly-typed Pydantic models.

In [ ]:
class Person(BaseModel):
    """Person information."""

    name: str = Field(description="Person's name")
    age: int = Field(description="Person's age")
    occupation: str = Field(description="Person's occupation")


pydantic_parser = PydanticOutputParser(pydantic_object=Person)

prompt = ChatPromptTemplate.from_template(
    "Generate a fictional person who is a {occupation}.\n{format_instructions}"
)

chain = prompt | llm | pydantic_parser
result = chain.invoke(
    {
        "occupation": "software engineer",
        "format_instructions": pydantic_parser.get_format_instructions(),
    }
)

print(f"Result: {result}")
print(f"Type: {type(result)}")
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Occupation: {result.occupation}")

## 5. Complex Pydantic Model

Parse into nested structures with multiple fields.

In [ ]:
class StoryAnalysis(BaseModel):
    """Story analysis."""

    main_characters: List[str] = Field(description="Main characters in the story")
    setting: str = Field(description="Where the story takes place")
    theme: str = Field(description="Primary theme")


story_parser = PydanticOutputParser(pydantic_object=StoryAnalysis)

prompt = ChatPromptTemplate.from_template(
    "Analyze this story:\n{story}\n\n{format_instructions}"
)

chain = prompt | llm | story_parser
result = chain.invoke(
    {
        "story": "Alice went to Wonderland and had tea with the Mad Hatter. It was a strange adventure about growing up.",
        "format_instructions": story_parser.get_format_instructions(),
    }
)

print(f"Characters: {result.main_characters}")
print(f"Setting: {result.setting}")
print(f"Theme: {result.theme}")

## 6. Chaining Multiple Parsers

Use different parsers in a workflow.

In [ ]:
# Step 1: Generate list of topics
list_prompt = ChatPromptTemplate.from_template(
    "List 3 {category}.\n{format_instructions}"
)
list_parser = CommaSeparatedListOutputParser()
list_chain = list_prompt | llm | list_parser

topics = list_chain.invoke(
    {
        "category": "animals",
        "format_instructions": list_parser.get_format_instructions(),
    }
)

print(f"Generated topics: {topics}\n")

# Step 2: Get JSON info for first topic
json_prompt = ChatPromptTemplate.from_template(
    "Provide brief info about {topic} as JSON with keys: name, habitat, diet.\n{format_instructions}"
)
json_parser = JsonOutputParser()
json_chain = json_prompt | llm | json_parser

info = json_chain.invoke(
    {"topic": topics[0], "format_instructions": json_parser.get_format_instructions()}
)

print(f"Info about {topics[0]}:")
print(f"  Habitat: {info.get('habitat')}")
print(f"  Diet: {info.get('diet')}")

## Summary

**Output Parser Patterns:**

| Parser | Output Type | Use Case |
|--------|-------------|----------|
| `StrOutputParser` | `str` | Simple text extraction |
| `CommaSeparatedListOutputParser` | `List[str]` | List generation |
| `JsonOutputParser` | `dict` | Flexible structured data |
| `PydanticOutputParser` | `BaseModel` | Type-safe structured data |

**Key Benefits:**
- **Type Safety**: Pydantic models provide validation
- **Format Instructions**: Parsers generate prompts automatically
- **Error Handling**: Built-in validation and parsing errors
- **Composability**: Mix parsers in chains
- **Documentation**: Models serve as data contracts

**When to Use:**
- **String Parser**: Simple text responses
- **List Parser**: Generating options, enumerations
- **JSON Parser**: Flexible data without strict schema
- **Pydantic Parser**: APIs, databases, strict validation